In [2]:
import glob
import pathlib
from torchvision.transforms import ToTensor, Normalize, Resize, ToPILImage
import PIL
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.data import Dataset
import torch
class ImageLoader(Dataset):
    def __init__(self, path='/media/i008/ssd500/small_sneaker_not_sneaker/sneaker'):
        super().__init__()
        
        self.path = path
        
        path = pathlib.Path(path)
        self.image_list = list(path.glob("*.jpg"))
        
    def __len__(self):
        return len(self.image_list)
    
    
    def __getitem__(self, ix):
        image = PIL.Image.open(self.image_list[ix]).convert("RGB")
        image = Resize((256, 256))(image)
        image = ToTensor()(image)
        
        return image


In [3]:
#     def forward(self, image_list, feature_maps):
#         grid_sizes = tuple([feature_map.shape[-2:] for feature_map in feature_maps])
#         image_size = image_list.tensors.shape[-2:]
#         strides = tuple((image_size[0] / g[0], image_size[1] / g[1]) for g in grid_sizes)
#         dtype, device = feature_maps[0].dtype, feature_maps[0].device
#         self.set_cell_anchors(dtype, device)
#         anchors_over_all_feature_maps = self.cached_grid_anchors(grid_sizes, strides)
#         anchors = []
#         for i, (image_height, image_width) in enumerate(image_list.image_sizes):
#             anchors_in_image = []
#             for anchors_per_feature_map in anchors_over_all_feature_maps:
#                 anchors_in_image.append(anchors_per_feature_map)
#             anchors.append(anchors_in_image)
#         anchors = [torch.cat(anchors_per_image) for anchors_per_image in anchors]
#         return anchors

In [4]:
from torchvision.models.detection.rpn import AnchorGenerator

In [91]:
anch = AnchorGenerator(sizes=((128,),(64,)), aspect_ratios=((1.0,),(1.0,)))

In [92]:
%pdb 1

from collections import namedtuple

t = namedtuple('bla', ['tensors','image_sizes'])

# 
ok = anch(image_list=t(torch.rand(1, 1), 
                       torch.tensor([[32,32],[32,32]])),
          
          feature_maps=[torch.rand(2, 1, 8, 8)])#, torch.rand(1,1,4,4)])

Automatic pdb calling has been turned ON


In [90]:
ok[2]

IndexError: list index out of range

> <ipython-input-90-1772b66b7dd9>(1)<module>()
----> 1 ok[2]

ipdb> q


In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [43]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
        )
  

In [ ]:
# class DenseBlock(nn.Block):
#     def __init__(self, num_convs, num_channels, **kwargs):
#         super(DenseBlock, self).__init__(**kwargs)
#         self.net = nn.Sequential()
#         for _ in range(num_convs):
#             self.net.add(conv_block(num_channels))

#     def forward(self, X):
#         for blk in self.net:
#             Y = blk(X)
#             # Concatenate the input and output of each block on the channel
#             # dimension
#             X = nd.concat(X, Y, dim=1)
#         return X

In [107]:
class Bla(torch.nn.Module):
    
        def __init__(self):
            super(Bla, self).__init__()
            self.m = torch.nn.Sequential(*[torch.nn.Conv2d(32, 32, 3) for a in range(6)])
            
        def forward(self, X):
            for m in self.m:
                Y = m(X)
                torch.cat([X, Y], dim=1)
                break

In [108]:
bla = Bla(to)

In [106]:
for m in bla.m:
    print(m)

Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))


In [109]:
bla(torch.rand(1, 3, 224, 224))

RuntimeError: Given groups=1, weight of size 32 32 3 3, expected input[1, 3, 224, 224] to have 32 channels, but got 3 channels instead

> /home/i008/anaconda3/envs/dl/lib/python3.7/site-packages/torch/nn/modules/conv.py(338)forward()
    336                             _pair(0), self.dilation, self.groups)
    337         return F.conv2d(input, self.weight, self.bias, self.stride,
--> 338                         self.padding, self.dilation, self.groups)
    339 
    340 

ipdb> q


In [110]:
from torchvision.models.densenet import densenet121

In [111]:
d = densenet121()

In [113]:
from torchsummary import summary

In [ ]:
a